<a href="https://colab.research.google.com/github/LohitSubodh/Guess-Word/blob/main/hangman_api_user.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [ ]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# function which returns a dictionary containing n-grams and their frequency
def generate_N_grams(words, n):
    # Create an empty dictionary to store ngrams
    ngrams = {}
    
    # Loop through each word in the input list
    for word in words:
        # If the length of the word is less than the value of 'n', skip to the next iteration
        if len(word) < n:
            continue
        
        # Loop through each character in the word up to 'n' characters from the end
        for i in range(len(word)-n+1):
            # Extract an 'n'-gram from the word based on the current iteration
            ngram = word[i:i+n]
            
            # If the 'n'-gram is not already in the dictionary, add it with a value of 1
            if ngram not in ngrams:
                ngrams[ngram] = 1
            # If the 'n'-gram is already in the dictionary, increment its value by 1
            else:
                ngrams[ngram] += 1
    
    # Return the dictionary containing the 'n'-grams and their frequency counts
    return ngrams


In [ ]:
def ngramprob(self, word, letter):
    total_count = 0  # total count of all n-grams of length n-1 that start with the same prefix as word
    underscore_index = word.find("_")  # index of the underscore in the word

    # iterate over all lowercase letters
    for char in string.ascii_lowercase:  
        try:
            # add count of n-gram to total_count if it exists in n-grams dictionary
            total_count += self.ngrams[len(word)-1][word[:underscore_index] + char + word[underscore_index+1:]]
        except KeyError:
            pass
    
    # return the probability of the letter given the previous n-gram, calculated as follows:
    # count of n-gram with letter at underscore position divided by total count of all n-grams with the same prefix
    return self.ngrams[len(word)-1][word[:underscore_index] + letter + word[underscore_index+1:]] / total_count


In [ ]:
HANGMAN_URL = "https://sg.trexsim.com/trexsim/hangman"

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        self.verbose = False
        full_dictionary_location = "/content/gdrive/My Drive/words_250000_train.txt"
        #self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        #self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        #self.current_dictionary = []
        text_file = open(full_dictionary_location,"r")
        words = text_file.read().splitlines()
        text_file.close()
        self.ngrams=[generate_N_grams(words,i) for i in range(1,6)]

    def ngramprob(self, word, letter):
      total_count = 0  # total count of all n-grams of length n-1 that start with the same prefix as word
      underscore_index = word.find("_")  # index of the underscore in the word

      # iterate over all lowercase letters
      for char in string.ascii_lowercase:  
          try:
              # add count of n-gram to total_count if it exists in n-grams dictionary
              total_count += self.ngrams[len(word)-1][word[:underscore_index] + char + word[underscore_index+1:]]
          except KeyError:
              pass
      
      # return the probability of the letter given the previous n-gram, calculated as follows:
      # count of n-gram with letter at underscore position divided by total count of all n-grams with the same prefix
      return self.ngrams[len(word)-1][word[:underscore_index] + letter + word[underscore_index+1:]] / total_count


    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        word=word[::2]
        available = list(set(string.ascii_lowercase) - set(self.guessed_letters))
        probs=[]
        contributions=[0,0,0,0,0,0]
        for char in available:
            char_prob=0
            for i in range(len(word)):
                if word[i]!="_":
                    continue
                # find the longest continious string without any "_"
                # 2gram
                try: # a _
                    if word[i-1]!="_":
                        contributions[1]+=self.ngramprob(word[i-1:i+1],char)
                        char_prob+=self.ngramprob(word[i-1:i+1],char)
                except KeyError:
                    pass
                try: # _ a
                    if word[i+1]!="_":
                        contributions[1]+=self.ngramprob(word[i:i+2],char)
                        char_prob+=self.ngramprob(word[i:i+2],char)
                except KeyError:
                    pass
                # 3gram
                try: # a b _
                    if word[i-1]!="_" and word[i-2]!="_":
                        contributions[2]+=self.ngramprob(word[i-2:i+1],char)
                        char_prob+=self.ngramprob(word[i-2:i+1],char)
                except KeyError:
                    pass
                try:
                    # a _ b
                    if word[i+1]!="_" and word[i-1]!="_":
                        contributions[2]+=self.ngramprob(word[i-1:i+2],char)
                        char_prob+=self.ngramprob(word[i-1:i+2],char)
                except KeyError:
                    pass
                try: # _ a b 
                    if word[i+1]!="_" and word[i+2]!="_":
                        contributions[2]+=self.ngramprob(word[i:i+3],char)
                        char_prob+=self.ngramprob(word[i:i+3],char)
                except KeyError:
                    pass
                # 4gram
                try: # a b c _
                    if word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_":
                        contributions[3]+=self.ngramprob(word[i-3:i+1],char)
                        char_prob+=self.ngramprob(word[i-3:i+1],char)
                except KeyError:
                    pass
                try: # a b _ c
                    if word[i+1]!="_" and word[i-1]!="_" and word[i-2]!="_":
                        contributions[3]+=self.ngramprob(word[i-2:i+2],char)
                        char_prob+=self.ngramprob(word[i-2:i+2],char)
                except KeyError:
                    pass
                try: # a _ b c
                    if word[i+1]!="_" and word[i+2]!="_" and word[i-1]!="_":
                        contributions[3]+=self.ngramprob(word[i-1:i+3],char)
                        char_prob+=self.ngramprob(word[i-1:i+3],char)
                except KeyError:
                    pass

                try:  # _ a b c
                    if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_":
                        contributions[3]+=self.ngramprob(word[i:i+4],char)
                        char_prob+=self.ngramprob(word[i:i+4],char) 
                except KeyError:
                    pass
                # 5 gram
                try: # a b c d _
                    if word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_" and word[i-4]!="_" :
                        contributions[4]+=self.ngramprob(word[i-4:i+1],char)
                        char_prob+=self.ngramprob(word[i-4:i+1],char)
                except KeyError:
                    pass
                try: # a b c _ d
                    if word[i+1]!="_" and word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_" :
                        contributions[4]+=self.ngramprob(word[i-3:i+2],char)
                        char_prob+=self.ngramprob(word[i-3:i+2],char)
                except KeyError:
                    pass
                try: # a b _ c d
                    if word[i+1]!="_" and word[i+2]!="_" and word[i-1]!="_" and word[i-2]!="_" :
                        contributions[4]+=self.ngramprob(word[i-2:i+3],char)
                        char_prob+=self.ngramprob(word[i-2:i+3],char)
                except KeyError:
                    pass
                try: # a _ b c d
                    if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_" and word[i-1]!="_" :
                        contributions[4]+=self.ngramprob(word[i-1:i+4],char)
                        char_prob+=self.ngramprob(word[i-1:i+4],char)
                except KeyError:
                    pass
                try: # _ a b c d
                    if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_" and word[i+4]!="_" :
                        contributions[4]+=self.ngramprob(word[i:i+5],char)
                        char_prob+=self.ngramprob(word[i:i+5],char)
                except KeyError:
                    pass
                # 6 gram
                try: # a b c d e _
                    if word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_" and word[i-4]!="_" and word[i-5]!="_" :
                        contributions[5]+=self.ngramprob(word[i-5:i+1],char)
                        char_prob+=self.ngramprob(word[i-5:i+1],char)
                except KeyError:
                    pass
                try: # a b c d _ e
                    if word[i+1]!="_" and word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_" and word[i-4]!="_" :
                        contributions[5]+=self.ngramprob(word[i-4:i+2],char)
                        char_prob+=self.ngramprob(word[i-4:i+2],char)
                except KeyError:
                    pass
                try: # a b c _ d e
                    if word[i+1]!="_" and word[i+2]!="_" and word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_" :
                        contributions[5]+=self.ngramprob(word[i-3:i+3],char)
                        char_prob+=self.ngramprob(word[i-3:i+3],char)
                except KeyError:
                    pass
                try: # a b _ c d e
                    if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_" and word[i-1]!="_" and word[i-2]!="_" :
                        contributions[5]+=self.ngramprob(word[i-2:i+4],char)
                        char_prob+=self.ngramprob(word[i-2:i+4],char)
                except KeyError:
                    pass
                try: # a _ b c d e
                    if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_" and word[i+4]!="_" and word[i-1]!="_" :
                        contributions[5]+=self.ngramprob(word[i-1:i+5],char)
                        char_prob+=self.ngramprob(word[i-1:i+5],char)
                except KeyError:
                    pass
                try: # _ a b c d e
                   if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_" and word[i+4]!="_" and word[i+5]!="_" :
                        contributions[5]+=self.ngramprob(word[i:i+6],char)
                        char_prob+=self.ngramprob(word[i:i+6],char)
                except KeyError:
                    pass

                # 1gram _
                char_prob+=self.ngramprob(word[i:i+1],char)
                contributions[0]+=self.ngramprob(word[i:i+1],char)
            probs.append(char_prob)
        # create sorted dictionary of probabilities
        probs_dict=dict(zip(available,probs))
        probs_sorted=sorted(probs_dict.items(), key=lambda x: x[1], reverse=True)
        if self.verbose:
            # iterate on probs_sorted
            for i in probs_sorted:
                print(i[0],round(i[1],2),end="\n")
            print("contribution of each letter")
            print(contributions)
        return available[probs.index(max(probs))]


    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary


    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        #self.current_dictionary = self.full_dictionary
                        
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
              
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    HANGMAN_URL + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
              try:
                    self.message = result["error_msg"]
              except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

In [ ]:

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")
        
        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        guess_letter = '!'
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter
   


In [ ]:
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [ ]:
api = HangmanAPI(access_token="71c4a67125f2a202b642302573bac2", timeout=2000)


In [ ]:
%pwd

'/content'

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [ ]:
for i in range(50):
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: 245d85c20fdf. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '245d85c20fdf', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ '}
Guessing letter: i
Sever response: {'game_id': '245d85c20fdf', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ i _ _ i _ _ '}
Guessing letter: a
Sever response: {'game_id': '245d85c20fdf', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ i _ _ i _ _ '}
Guessing letter: n
Sever response: {'game_id': '245d85c20fdf', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ i _ _ i n _ '}
Guessing letter: o
Sever response: {'game_id': '245d85c20fdf', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ o i _ _ i n _ '}
Guessing letter: t
Sever response: {'game_id': '245d85c20fdf', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ o i _ _ i n _ '}
Guessing letter: s
Sever response: {'game_id': '245d85c20fdf', 'status': 'ongoing', 'tries_remains': 2, '

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)